# Creating municipality RIDA layers

Assuming that the data we received from Septima by WeTransfer (2 folders with fgb files, plus one separate fgb file) is in the folder `data/RIDA` (from Dropbox [here](https://www.dropbox.com/scl/fo/38gkcvyfex0atdbqe3hfn/h?rlkey=z9xwq2mh6ga03c301b7f66wqx&dl=0))

This script imports each of the fgb layers we need and cuts them into muncipality-sized bites.

In [13]:
# import packages
import geopandas as gpd
import os
import matplotlib.pyplot as plt

In [2]:
# read in kommuner data and make subfolders

muni = gpd.read_file("data/raw/municipalities/muni_boundary.gpkg")
muni.head()

,id.namespace,id.lokalId,status,geometristatus,virkningFra,virkningTil,virkningsaktoer,forretningshaendelse,registreringFra,registreringTil,...,landekode,skala,udtraeksdato,kommunekode,LAU1vaerdi,udenforKommuneinddeling,regionskode,regionsLokalId,region,geometry
0,http://data.gov.dk/dagi,389148,vedtaget,endelig,2022-01-01,NaT,Økonomi- og Indenrigsministeriet,ændretBekendtgørelse,2021-12-14 09:46:41,NaT,...,DK,1:10.000,2022-02-04 14:50:06,0370,370,false,1085,389100,NaN,"MULTIPOLYGON (((672902.450 6143708.870, 672880..."
1,http://data.gov.dk/dagi,389144,vedtaget,endelig,2022-01-01,NaT,Geodatastyrelsen,ændretBekendtgørelse,2021-12-14 09:46:41,NaT,...,DK,1:10.000,2022-02-04 14:50:06,0330,330,false,1085,389100,NaN,"MULTIPOLYGON (((654675.250 6154223.210, 654636..."
2,http://data.gov.dk/dagi,389178,vedtaget,endelig,2021-02-19,NaT,Økonomi- og Indenrigsministeriet,fejlrettelseØvrige,2021-02-19 09:03:56,NaT,...,DK,1:10.000,2022-02-04 14:50:06,0766,766,false,1082,389101,NaN,"MULTIPOLYGON (((563379.450 6175110.360, 563346..."
3,http://data.gov.dk/dagi,389166,vedtaget,endelig,2022-01-01,NaT,Økonomi- og Indenrigsministeriet,ændretBekendtgørelse,2021-12-14 10:23:11,NaT,...,DK,1:10.000,2022-02-04 14:50:06,0510,510,false,1083,389102,NaN,"MULTIPOLYGON (((516449.610 6137370.130, 516423..."
4,http://data.gov.dk/dagi,389168,vedtaget,endelig,2022-01-01,NaT,Økonomi- og Indenrigsministeriet,ændretBekendtgørelse,2021-12-14 10:23:11,NaT,...,DK,1:10.000,2022-02-04 14:50:06,0550,550,false,1083,389102,NaN,"MULTIPOLYGON (((482562.400 6122027.620, 482507..."


<img src="struct.png" width=60%>


In [3]:
for _, row in muni.iterrows():
    os.makedirs(f"data/raw/{row.kommunekode}", exist_ok=True)

In [32]:
rida_dict = {
    # "facilit_indkoeb.fgb": "indkoeb.gpkg",
    # "land_fortid.fgb": "fortidsminder.gpkg"
}

In [33]:
%%time
for k, v in rida_dict.items():
    gdf = gpd.read_file(f"data/RIDA/{k}")
    assert gdf.crs == muni.crs
    for _, row in muni.iterrows():
        gdf_curr = gdf[gdf.intersects(row.geometry)].copy()
        gdf_curr = gdf_curr.reset_index(drop=True)
        if not gdf_curr.empty:
            gdf_curr.to_file(f"data/raw/{row.kommunekode}/{v}")    

CPU times: user 5.12 s, sys: 901 ms, total: 6.02 s
Wall time: 6.27 s


### PICK UP HERE:

* add all (needed) fgb files to rida_dict
* run to create separate layers, PUSH TO MAIN SEPARATELY!
* add a code to save kommune polygons separately
* adjust 00 file to importing data by kommunekode (config file)
* move 03a to archive
* edit 03b, convert to 03a, adding new layers!
* check if script works